# 데이터 모델링
* 크롤링 데이터
* 강의명 데이터
* 인구통계학적 데이터?

## 필요 라이브러리 로드

In [1]:
!pip install torch
!pip install transformers
!pip install pycaret


import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torch
from transformers import get_linear_schedule_with_warmup
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
import pycaret
from pycaret.classification import *

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder


In [3]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

## 사용할 데이터 로드

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
courses_df = pd.read_csv('/content/drive/MyDrive/PROJECT/교육데이터분석공모전/데이터분석/모델링_예측/merged_lecture.csv')
crawling_df = pd.read_csv('/content/drive/MyDrive/PROJECT/교육데이터분석공모전/데이터분석/모델링_예측/크롤링 데이터/크롤링_전처리.csv')

In [6]:
#courses_df = pd.read_csv('./merged_lecture_3.csv')
#crawling_df = pd.read_csv('./크롤링 데이터/크롤링_전처리.csv')

In [5]:
courses_df.head()

,강의 명,강의 카테고리명
0,영어로배우는 키성장요가,사회/교양
1,창의쑥쑥키즈페인팅,사회/교양
2,건강튼튼 어린이발레,사회/교양
3,팝송으로 배우는 잉글리쉬,사회/교양
4,시니어 모델워킹,사회/교양


In [6]:
crawling_df.head()

,검색어,출처,제목,설명,링크
0,중장년 취미 생활,뉴스,성북구 중장 1 인 가구 힐링 원데이 클래스,향후 성북구 고려대학교 KU 건강 연구소 해 우울 감 을 경험 하는 중장 집단 심리...,https://n.news.naver.com/mnews/article/081/000...
1,중장년 취미 생활,뉴스,성북구 중장 1 인 가구 셀프 집 수리 힐링 원데이 클래스,성북구 향후 고려대학교 KU 건강 연구소 해 우울 감 을 경험 하는 중장 집단 심리...,https://n.news.naver.com/mnews/article/421/000...
2,중장년 취미 생활,뉴스,성북구 중장 1 인 가구 맞춤 야간,내년 에는 동아리 을 하는 1 인 가구 생활 뿐 만 건강 까지 챙긴다는 이다 이승 ...,https://n.news.naver.com/mnews/article/629/000...
3,중장년 취미 생활,뉴스,quot 미래 일단 잡자 quot MZ 세대 겨냥 이색 보험 뜬다,윤씨 자산 중장 년층 비해 부족하지만 하고 싶은 일 많은 20 대들을 위 보험 을 ...,https://www.greened.kr/news/articleView.html?i...
4,중장년 취미 생활,뉴스,전북 비정규직 인원 3만 명대 육박 프리터 족 급증,회사 다닐 보다 시간 구애 받지 않고 자유로운 여가 생활 을 즐기는 여유 많아져 관...,http://www.domin.co.kr/news/articleView.html?i...


In [7]:
crawling_df['제목'] = crawling_df['제목'].astype(str)
crawling_df['설명'] = crawling_df['설명'].astype(str)


In [8]:
# 1. 데이터 전처리
import re
import time
start_time = time.time()

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z가-힣0-9\s]', '', text)  # 특수문자 제거
    text = text.lower()  # 영어 소문자 변환
    return text

courses_df['강의 명'] = courses_df['강의 명'].apply(preprocess_text)
crawling_df['제목'] = crawling_df['제목'].apply(preprocess_text)
crawling_df['설명'] = crawling_df['설명'].apply(preprocess_text)

preprocessing_time = time.time() - start_time
# 전처리 시간 측정

print(f"전처리 시간: {preprocessing_time:.2f}초")

전처리 시간: 0.31초


In [9]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
# BERT 모델 및 토크나이저 로드
# BERT 모델 로드
start_time = time.time()
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')
model_loading_time = time.time() - start_time
print(f"BERT 모델 로딩 시간: {model_loading_time:.2f}초")


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BERT 모델 로딩 시간: 13.47초


In [10]:
# 문장 임베딩 생성 함수
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings

In [ ]:
# 2. BERT 모델을 이용한 텍스트 임베딩
start_time = time.time()

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

courses_df['임베딩'] = courses_df['강의 명'].apply(get_bert_embeddings)
crawling_df['임베딩'] = (crawling_df['제목'] + ' ' + crawling_df['설명']).apply(get_bert_embeddings)

embedding_time = time.time() - start_time
print(f"embedding 시간: {embedding_time:.2f}초")

In [ ]:
# 3. 관심사 기반 가중치 적용
def apply_interest_weights(embeddings, interests, weights):
    interest_embeddings = [get_bert_embeddings(interest) for interest in interests]
    weighted_embeddings = sum(weight * embedding for weight, embedding in zip(weights, interest_embeddings))
    return cosine_similarity(embeddings, weighted_embeddings.reshape(1, -1))

In [ ]:
# 4. 추천 시스템 구현
def recommend_courses(interests, weights):
    course_embeddings = np.vstack(courses_df['임베딩'].values)
    scores = apply_interest_weights(course_embeddings, interests, weights)
    courses_df['유사도'] = scores
    return courses_df.sort_values(by='유사도', ascending=False)

In [ ]:
# 예제 관심사 입력 및 가중치
interests = ['창업', '디자인']
weights = [0.7, 0.3]

# 추천 결과
start_time = time.time()
recommendations = recommend_courses(interests, weights)
recommendation_time = time.time() - start_time

In [ ]:
# 각 처리 시간 출력
print(f"전처리 시간: {preprocessing_time:.2f}초")
print(f"텍스트 임베딩 시간: {embedding_time:.2f}초")
print(f"추천 시스템 실행 시간: {recommendation_time:.2f}초")

In [ ]:
# 추천 결과 출력
print(recommendations)

## top 5개 뽑기

In [ ]:
import time

# BERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()

# 강의명 임베딩
start_time = time.time()
course_embeddings = get_bert_embeddings(courses_df['강의 명'].tolist())
end_time = time.time()
print(f"강의명 임베딩 시간: {end_time - start_time} 초")

# 관심사 설명 임베딩
start_time = time.time()
interest_embeddings = get_bert_embeddings(crawling_df['설명'].tolist())
end_time = time.time()
print(f"관심사 설명 임베딩 시간: {end_time - start_time} 초")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 유사도 계산 함수
def recommend_courses(interest_embeddings, course_embeddings, top_k=3):
    recommendations = []
    for interest in interest_embeddings:
        similarities = cosine_similarity([interest], course_embeddings)[0]
        top_indices = similarities.argsort()[-top_k:][::-1]
        recommendations.append(top_indices)
    return recommendations

# 추천 결과 출력
recommendations = recommend_courses(interest_embeddings, course_embeddings)

for i, recs in enumerate(recommendations):
    print(f"관심사 {i+1}에 대한 추천 강의:")
    for idx in recs:
        print(f"- {courses_df.iloc[idx]['강의 명']}")

In [ ]:
# 크롤링 데이터 설명 임베딩
crawling_df['embedding'] = crawling_df['설명'].apply(lambda x: get_embedding(x))

# 강의명 임베딩
courses_df['embedding'] = courses_df['강의명'].apply(lambda x: get_embedding(x))

In [ ]:
# 추천 함수
def recommend_lectures(interest, top_n=5):
    interest_embedding = get_embedding(interest)
    similarities = []
    for i, row in lecture_df.iterrows():
        sim = cosine_similarity(interest_embedding, row['embedding'])
        similarities.append((row['강의명'], sim))

    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return [lecture for lecture, sim in similarities[:top_n]]

In [ ]:
# 관심사 입력 받기
interest = "창업, 디자인"
recommended_lectures = recommend_lectures(interest)

print("추천 강의 목록:")
for lecture in recommended_lectures:
    print(lecture)

## Konlpy 사용

In [ ]:
!pip install konlpy
import time
import re
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

courses_df = pd.read_csv('/content/drive/MyDrive/PROJECT/교육데이터분석공모전/데이터분석/모델링_예측/merged_lecture.csv')
crawling_df = pd.read_csv('/content/drive/MyDrive/PROJECT/교육데이터분석공모전/데이터분석/모델링_예측/크롤링 데이터/크롤링_전처리.csv')

In [ ]:
# 텍스트 전처리 함수
okt = Okt()



def preprocess_text(text):
    if not isinstance(text, str):
        return ''
    # 텍스트 정규화
    text = re.sub(r'[^a-zA-Z0-9가-힣\s]', '', text)  # 특수문자 제거
    text = okt.morphs(text)  # 형태소 분석

    return text  # 예시로 단순히 반환하는 코드
    return ' '.join(text)

# 전처리 시간 측정
start_time = time.time()
courses_df['강의 명'] = courses_df['강의 명'].apply(preprocess_text)
crawling_df['제목'] = crawling_df['제목'].apply(preprocess_text)
crawling_df['설명'] = crawling_df['설명'].apply(preprocess_text)
preprocessing_time = time.time() - start_time
print(f"전처리 시간: {preprocessing_time:.2f}초")


In [ ]:
import numpy as np
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()

# 강의 명과 설명을 하나의 텍스트로 결합
crawling_df['전체 텍스트'] = crawling_df['제목'] + " " + crawling_df['설명']
# NaN 값 제거
crawling_df['전체 텍스트'] = crawling_df['전체 텍스트'].fillna('')
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix_courses = vectorizer.fit_transform(courses_df['강의 명'])
tfidf_matrix_crawling = vectorizer.transform(crawling_df['전체 텍스트'])


In [ ]:
# 관심사 입력받기
interests = ["창업", "디자인"]

# 관심사 임베딩 생성
interest_embeddings = vectorizer.transform(interests)

# 맞춤형 강의 추천 함수
def recommend_courses(interest_embeddings, tfidf_matrix_courses, course_titles):
    similarities = cosine_similarity(interest_embeddings, tfidf_matrix_courses)
    recommended_courses = []
    for sim in similarities:
        recommended_courses.append(course_titles[np.argmax(sim)])
    return recommended_courses

# 추천 강의명 출력
recommended_courses = recommend_courses(interest_embeddings, tfidf_matrix_courses, courses_df['강의 명'])

print("추천 강의명:")
for course in recommended_courses:
    print(course)

In [ ]:
# 추천 시간 측정
start_time = time.time()
scores = recommend_courses(tfidf_matrix_courses, interest_embeddings, weights)
courses_df['추천 점수'] = scores
recommendation_time = time.time() - start_time
print(f"추천 시간: {recommendation_time:.2f}초")


In [ ]:
!pip install transformers
import time
import torch
from transformers import BertTokenizer, BertModel


# BERT 모델 로드
start_time = time.time()
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')
model_loading_time = time.time() - start_time
print(f"BERT 모델 로딩 시간: {model_loading_time:.2f}초")




In [ ]:
# 관심사 입력 및 가중치 부여 (예시)
interests = ['창업', 디자인']
weights = [0.7, 0.3]

# 텍스트 임베딩 함수
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

# 임베딩 생성 시간 측정
start_time = time.time()
courses_df['임베딩'] = courses_df['강의 명'].apply(get_embedding)
crawling_df['임베딩'] = (crawling_df['제목'] + " " + crawling_df['설명']).apply(get_embedding)
embedding_time = time.time() - start_time
print(f"임베딩 생성 시간: {embedding_time:.2f}초")

In [ ]:
# 관심사 임베딩 생성
interest_embeddings = [get_embedding(preprocess_text(interest)) for interest in interests]

# 맞춤형 강의 추천
def recommend_courses(course_embeddings, interest_embeddings, weights):
    recommendations = []
    for course_embedding in course_embeddings:
        score = 0
        for interest_embedding, weight in zip(interest_embeddings, weights):
            score += weight * torch.cosine_similarity(course_embedding.unsqueeze(0), interest_embedding.unsqueeze(0)).item()
        recommendations.append(score)
    return recommendations

In [ ]:
# 추천 점수 기준으로 정렬
recommended_courses = courses_df.sort_values(by='추천 점수', ascending=False)
print(recommended_courses)

## LDA 활용
* Latent Diriklet Allocation